In [1]:
!pip install twilio
import subprocess
import os
import cv2
import numpy as np
from twilio.rest import Client
from IPython.display import Video

try:
    # Run the ffmpeg command with the '-version' option
    result = subprocess.run(['ffmpeg', '-version'], capture_output=True, text=True)
    
    # Check the result
    if result.returncode == 0:
        print("ffmpeg is available.")
        print(result.stdout)  # Print the ffmpeg version and information
    else:
        print("ffmpeg is not available.")
        print(result.stderr)  # Print the error message if any
except FileNotFoundError:
    print("ffmpeg is not installed.")


ffmpeg is available.
ffmpeg version 6.0-full_build-www.gyan.dev Copyright (c) 2000-2023 the FFmpeg developers
built with gcc 12.2.0 (Rev10, Built by MSYS2 project)
configuration: --enable-gpl --enable-version3 --enable-static --disable-w32threads --disable-autodetect --enable-fontconfig --enable-iconv --enable-gnutls --enable-libxml2 --enable-gmp --enable-bzlib --enable-lzma --enable-libsnappy --enable-zlib --enable-librist --enable-libsrt --enable-libssh --enable-libzmq --enable-avisynth --enable-libbluray --enable-libcaca --enable-sdl2 --enable-libaribb24 --enable-libdav1d --enable-libdavs2 --enable-libuavs3d --enable-libzvbi --enable-librav1e --enable-libsvtav1 --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxavs2 --enable-libxvid --enable-libaom --enable-libjxl --enable-libopenjpeg --enable-libvpx --enable-mediafoundation --enable-libass --enable-frei0r --enable-libfreetype --enable-libfribidi --enable-liblensfun --enable-libvidstab --enable-libvmaf --enable-libzimg 

In [33]:
import cv2
import numpy as np
from twilio.rest import Client
from datetime import datetime  # Import the datetime module

#Twilio credentials
account_sid = ""
auth_token = ""

# Twilio phone number
twilio_phone_number = ""

# Recipient's phone number
recipient_phone_number = ""

twilio_client = Client(account_sid, auth_token)

# Load the video file
cap = cv2.VideoCapture(0)

# Define the codec for the output video file
fourcc = cv2.VideoWriter_fourcc(*'mp4v')

bgframe = None
# Flag to indicate motion detection
motion_detected = False

# Flag to indicate recording status
recording = False

# Variable for motion event count
big_motion_event_count = 0
motion_event_count = 0

while True:
    # Access the frame from the video
    success, img = cap.read()

    # Check if we get the frame
    if success:
        # Resize the frame because there is no need to process a large size frame
        img = cv2.resize(img, (800, 800))

        # BGR frame into grayscale format
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        # Avoid any change in pixel intensity
        gray = cv2.GaussianBlur(gray, (51, 51), 0)

        # Provide frame for first iteration as background (bgframe) is None
        if bgframe is None:
            bgframe = gray
            
        # Compute the difference between the background frame and current frame
        frameDelta = cv2.absdiff(bgframe, gray)

        # If any pixel value in 'frameDelta' is greater than 25, motion is detected
        _, thresh = cv2.threshold(frameDelta, 50, 255, cv2.THRESH_BINARY)

        # Create a kernel of 7x7
        kernel = np.ones((7, 7), np.uint8)

        # Dilate the frame
        thresh = cv2.dilate(thresh, None, iterations=4)

        # Find the contours
        contours, hierarchy = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        # Check for motion
        motion_detected = len(contours) > 0

        # Start or stop recording based on motion detection
        if motion_detected and not recording:
            recording = True
            motion_event_count += 1
            area = cv2.contourArea(contours[0]) if contours else 0

            if area > 1000:
                video_path = os.path.join("recorded_footage", f"Big Motion{motion_event_count}.mp4")
                motion_type = "BIG MOTION"
            else:
                video_path = os.path.join("recorded_footage", f"Small Motion{motion_event_count}.mp4")
                motion_type = "SMALL MOTION"
            out = cv2.VideoWriter(video_path, fourcc, 20.0, (800, 800))

            # Send an SMS notification when motion is detected and recording starts
            try:
                timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                sms_body = f"{motion_type} DETECTED! Motion video recorded at {timestamp}"
                message = twilio_client.messages.create(
                    body=sms_body,
                    from_=twilio_phone_number,
                    to=recipient_phone_number
                )
                print("SMS notification sent successfully!")
            except Exception as e:
                print(f"SMS notification sending failed: {str(e)}")

        elif not motion_detected and recording:
            recording = False
            out.release()

        for contour in contours:
            # Finding area of contour
            area = cv2.contourArea(contour)

            # If area is greater than the specified value, consider it as motion
            if area > 500:
                # Find the rectangle coordinates
                x, y, w, h = cv2.boundingRect(contour)

                # Draw the rectangle to indicate the moving object
                cv2.rectangle(img, (x, y), (x + w, y + h), (255, 0, 255), 3)
                cv2.putText(img, f"Time: {timestamp}", (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

        # Display the frame
        cv2.imshow('VIDEO', img)

        # Write the frame to the output video file if recording is enabled
        if recording:
            out.write(img)

        # Check for key press
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break

    else:
        break

# Release the resources
cap.release()
out.release()
cv2.destroyAllWindows()


SMS notification sending failed: HTTP 400 error: Unable to create record: Message body is required.
SMS notification sending failed: HTTP 400 error: Unable to create record: Message body is required.
SMS notification sending failed: HTTP 400 error: Unable to create record: Message body is required.
SMS notification sending failed: HTTP 400 error: Unable to create record: Message body is required.
SMS notification sending failed: HTTP 400 error: Unable to create record: Message body is required.
SMS notification sending failed: HTTP 400 error: Unable to create record: Message body is required.


In [34]:
import os
import subprocess

# Replace 'input_directory' with the current directory
input_directory = 'recorded_footage'
# Replace 'output_directory' with the current directory or a subdirectory within it
output_directory = 'recorded_footage_for_view'

# Check if the output directory exists, if not, create it
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

# Function to convert a single file from MP4 to OGG with improved resolution
def convert_mp4_to_ogg(input_file, output_file):
    # Customize video and audio settings for better resolution
    cmd = ['ffmpeg', '-i', input_file, '-c:v', 'libtheora', '-q:v', '8', '-c:a', 'libvorbis', '-q:a', '6', output_file]
    subprocess.call(cmd)

# Get a list of all MP4 files in the input directory
mp4_files = [file for file in os.listdir(input_directory) if file.endswith('.mp4')]

# Loop through the MP4 files and convert each one to OGG format
for mp4_file in mp4_files:
    mp4_path = os.path.join(input_directory, mp4_file)
    ogg_file = mp4_file[:-4] + '.ogg'  # Remove the '.mp4' extension and add '.ogg' extension
    ogg_path = os.path.join(output_directory, ogg_file)
    convert_mp4_to_ogg(mp4_path, ogg_path)

print("Conversion completed!")


Conversion completed!


In [1]:
import webbrowser
import subprocess

def open_html_file_in_chrome_tab(file_path):
    try:
        chrome_path = "C:/Program Files (x86)/Google/Chrome/Application/chrome.exe"
        subprocess.run([chrome_path, file_path])
    except FileNotFoundError:
        print("Chrome executable not found")

html_file_path = "file:///C:/Users/User/FYP%20DASHBOARD/newdashboard.html"
open_html_file_in_chrome_tab(html_file_path)


Chrome executable not found
